# Proximity analysis
In this notebook we will be working with data from OpenStreetMap to do proximity analysis to various urban amenities.

In [2]:
import folium
import geopandas
import pyproj
from shapely.ops import transform

## Import data

In [4]:
file_name = "data/Tampere-food_sources.geojson"

In [5]:
tampere_food_sources = geopandas.read_file(file_name)

In [6]:
tampere_food_sources.head()

,fid,osm_id,code,fclass,name,geometry
0,9,33403046,2511,convenience,K-Market Haapalinna,MULTIPOINT (2636757.686 8743361.457)
1,15,59359661,2501,supermarket,Nekalan Suurjako,MULTIPOINT (2647179.105 8736781.591)
2,69,257534920,2501,supermarket,Sale,MULTIPOINT (2645129.324 8737739.384)
3,74,269693256,2501,supermarket,S-market Pendoliino,MULTIPOINT (2646677.589 8741069.503)
4,121,442453872,2511,convenience,K-Market Kissanmaa,MULTIPOINT (2651590.496 8741284.592)


## Create buffers

In [7]:
tampere_food_sources_buffered = tampere_food_sources.buffer(1000)

## Dissolve buffers

In [8]:
tampere_food_sources_buffered_union = tampere_food_sources_buffered.unary_union
type(tampere_food_sources_buffered_union)

shapely.geometry.multipolygon.MultiPolygon

## Convert coordinates
We need to change the coordinate reference system in order to display the data correctly in Foliu

In [9]:
tampere_food_sources_4326 = tampere_food_sources.to_crs(epsg='4326')
tampere_food_sources_buffered_4326 = tampere_food_sources_buffered.to_crs(epsg='4326')

In [10]:
tampere_food_sources_4326_json = tampere_food_sources_4326.to_json()
tampere_food_sources_buffered_4326_json = tampere_food_sources_buffered_4326.to_json()

In [11]:
tampere_food_sources_buffered_union

projection = pyproj.Transformer.from_proj(
    pyproj.Proj(init='epsg:3857'),
    pyproj.Proj(init='epsg:4326')
)

tampere_food_sources_buffered_union_4326 = transform(projection.transform, tampere_food_sources_buffered_union)

/home/brylie/.cache/pypoetry/virtualenvs/suds-notebooks-maDYyLUp-py3.8/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/brylie/.cache/pypoetry/virtualenvs/suds-notebooks-maDYyLUp-py3.8/lib/python3.8/site-packages/pyproj/crs/crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
/home/brylie/.cache/pypoetry/virtualenvs/suds-notebooks-maDYyLUp-py3.8/lib/python3.8/site-packages/pyproj/c

In [17]:
tampere_food_sources_buffered_union_4326_json = geopandas.GeoSeries([tampere_food_sources_buffered_union_4326]).to_json()

## Create map
Next, we will display a map centered around Tampere.

In [13]:
tampere_coordinates = [61.4978, 23.7610]

In [14]:
map = folium.Map(tampere_coordinates)

## Add data to map
Now we can add the food sources data to the map.

In [18]:
folium.GeoJson(
    tampere_food_sources_4326_json,
    name="Tampere food sources",
).add_to(map)

folium.GeoJson(
    tampere_food_sources_buffered_union_4326_json,
    name="Tampere food buffers",
).add_to(map)

folium.LayerControl().add_to(map)

In [19]:
map